### This scripts transforms and loads the Enhanced Loan-Level Data csv files to the database.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
import configparser

# Function to drop entirely empty columns
def drop_empty_columns(df):
    return df.dropna(axis=1, how='all')

# Read database credentials and file directory from a config file
config = configparser.ConfigParser()
config.read('config.ini')

# Database connection URL
db_connection_url = f"postgresql+psycopg2://{config['database']['user']}:{config['database']['password']}@{config['database']['host']}:{config['database']['port']}/{config['database']['database']}"

# Create a SQLAlchemy engine
engine = create_engine(db_connection_url)

# Directory where your CSV files are stored
csv_folder_path = config['files']['enhanced_loan_level_dir']

# Iterate over each file in the folder and insert data into the database
for csv_file in os.listdir(csv_folder_path):
    if csv_file.endswith('.csv'):
        file_path = os.path.join(csv_folder_path, csv_file)

        # Use Pandas to load the CSV file
        df = pd.read_csv(file_path)

        # Drop columns that are entirely empty
        df = drop_empty_columns(df)

        # Use 'to_sql' to insert the data into the database, it creates a table if it does not exist
        df.to_sql('enhanced_loan_level_dir', engine, if_exists='append', index=False)

print("Loading data completed. All empty columns were dropped.")
